# Recommeded Retail Price
This notebook aims to create new features using our "recommendedRetailPrice"
**Spoiler:** I failed...

In [11]:
import numpy as np
import pandas as pd
from utils import read_data, process_time, merge_data, promo_detector, promo_detector_fixed, promotionAggregation, dataset_builder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import sys
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime

NUMBER_OF_LAGS = 4

sys.path.append("../../main/datasets/")
!ls  ../../main/datasets/

1.0v.zip


<hr>

## Defining metrics

Baseline_score function

In [12]:
def baseline_score(prediction, target, simulatedPrice):
    prediction = prediction.astype(int)

    return np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice)

Evaluation Metric

In [13]:
def feval(prediction, dtrain):
    
    prediction = prediction.astype(int)
    target = dtrain.get_label()

    simulatedPrice = dtrain.get_weight()
    
    return 'feval', np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice), True

<hr>

## Building our dataset
This notebook makes this step cleaner than the previous versions. So It'll be tidier and shorter than before!

In [14]:
infos, items, orders = read_data("../../main/datasets/")
print("Sanity checks...", infos.shape, items.shape, orders.shape)

Sanity checks... (10463, 3) (10463, 8) (2181955, 5)


In [15]:
# Changing our time signatures
process_time(orders)

In [16]:
df = dataset_builder(orders, items)

<hr>

## Feature building

**Last week salesPrice**

In [85]:
salesPriceMeanByItemAndWeek = orders.groupby(['itemID', 'group_backwards']).agg({'salesPrice':'mean'})
aux = pd.merge(df, salesPriceMeanByItemAndWeek, left_on=['itemID', 'group_backwards'], right_on=['itemID', 'group_backwards'], how='left')
aux = aux.groupby(['itemID', 'group_backwards']).agg(salesPriceLastWeek=('salesPrice','mean')).shift(-1)
df = pd.merge(df, aux, left_on=['itemID', 'group_backwards'], right_on=['itemID', 'group_backwards']).fillna(0)

**Recommended Retail Price ratio feature**

In [83]:
df['retailRatio'] = df['salesPriceLastWeek'] / df['recommendedRetailPrice']